# Learning Quantum Optics
## A Notebook for learning how to work with the package [QuantumOptics.jl](https://qojulia.org/)
### Florian Gahbauer
### February 25, 2021
### Revised March 6, 2021

In [1]:
using LinearAlgebra
using QuantumOptics
using WignerSymbols
using Plots
plotly()


Plots.PlotlyBackend()

## Creating Bases
In this section, we will test the SpinBasis function to create a spin-1 basis for the electron orbital angular momentum (S1) and a spin-1/2 basis for the electron spin (S2). 
Then, we will create a composite basis to describe a Hydrogen atom in the 2p state from these two bases. 

In [2]:
m1basis=SpinBasis(1//1)                         # basis for orbital angular momentum S1=L=|S1,m₁> with spin-1. 
m2basis=SpinBasis(1//2)                         # basis for spin S2=|S2,m₂> with spin-1/2  |↑>=[1; 0], |↓>=[0; 1] 
m1m2basis=m2basis ⊗ m1basis
#m1m2basis=CompositeBasis(m2basis,m1basis)       # Composite basis S1 ⊗ S2 (outer product)
#hbasis=tensor(sbasis,ibasis) # same as above, I think.
# We will call this the |L,S,mₗ,mₛ> basis or |mₗ,mₛ> for short, because we can describe it 
#by the z-component of each of the spins: |1↑>, |1↓>, |0↑>, |0↓>, |-1↑>, |-1↓> 

[Spin(1/2) ⊗ Spin(1)]

## Operators in the two bases
Next we will create some basic operators defined for the spin-1 and the spin-1/2 bases. 
We want the $L_z^{\frac{1}{2} \otimes 1}$ and $S_z^{\frac{1}{2} \otimes 1}$operator and the identity operator in each basis. 
To make life easier, we will set $\hbar=1$.

In [3]:
m1basisSz=0.5*sigmaz(m1basis)               # Sz operator for S1 basis
m2basisSz=0.5*sigmaz(m2basis)               # Sz operator for S2 basis: 0.5 \hbar [1 0; 0 -1]
m1basisSplus=sigmap(m1basis)                # Splus operator for S1 basis
m1basisSminus=sigmam(m1basis)               # Sminus operator for S1 basis
m2basisSplus=sigmap(m2basis)                # Splus operator for S2 basis: \hbar [0 1; 0  0]
m2basisSminus=sigmam(m2basis)               # Sminus operator for S1 basis: \hbar [0 0; 1  0]
m1basisIdentity=identityoperator(m1basis)   # Identity operator for S1 basis
m2basisIdentity=identityoperator(m2basis)   # Identity operator for S2 basis: [1 0; 0 1]
m1basisSS=0.75*m1basisIdentity              # S^2 operator for S1 basis
m2basisSS=0.75*m2basisIdentity              # S^2 operator for S2 basis: 0.75 \hbar^2 [1 0; 0 1]

Operator(dim=2x2)
  basis: Spin(1/2)
  [1, 1]  =  0.75+0.0im
  [2, 2]  =  0.75+0.0im

## Operators in the spin(1/2) $\otimes$ spin(1) basis
Now we need to express these operators in the spin(1/2) $\otimes$ spin(1) basis. The key is the [outer product](https://en.wikipedia.org/wiki/Outer_product). 



In [4]:
S1z = m2basisIdentity ⊗ m1basisSz           # S1z in S1S2 basis: 
S2z = m2basisSz ⊗ m1basisIdentity           # S2z in S1S2 basis: 
S1plus = m2basisIdentity ⊗ m1basisSplus     # S1plus in S1S2 basis 
S1minus = m2basisIdentity ⊗ m1basisSminus   # S1minus in S1S2 basis
S2plus = m2basisSplus ⊗ m1basisIdentity     # S2plus in S1S2 basis 
S2minus = m2basisSminus ⊗ m1basisIdentity   # S2minus in S1S2 basis
S1S2=S1z*S2z+0.5*(S2plus*S1minus+S2minus*S1plus)    # S1̇ S2 in S1S2 basis
#See eq. (E-19) of Chapter XII.3.b. of Cohen-Tannoudji Quantum Mechanics for S=S1, I=S2


Operator(dim=6x6)
  basis: [Spin(1/2) ⊗ Spin(1)]
  [1, 1]  =  0.5+0.0im
  [2, 2]  =  -0.5+0.0im
  [3, 2]  =  0.707107+0.0im
  [2, 3]  =  0.707107+0.0im
  [5, 4]  =  0.707107+0.0im
  [4, 5]  =  0.707107+0.0im
  [5, 5]  =  -0.5+0.0im
  [6, 6]  =  0.5+0.0im

In [5]:
# Now define S^2 in the S1S2 basis. 
# See eq (B-18) of Chapter X of Cohen-Tannoudji Quantum Mechanics
# Should be \hbar^2 [2 0 0 0; 0 1 1 0; 0 1 1 0; 0 0 0 2]
SS=0.75*identityoperator(m1m2basis) + 0.75*identityoperator(m1m2basis) + 2*S1z*S2z +S1plus*S2minus +S1minus*S2plus 

Operator(dim=6x6)
  basis: [Spin(1/2) ⊗ Spin(1)]
  [1, 1]  =  2.5+0.0im
  [2, 2]  =  0.5+0.0im
  [3, 2]  =  1.41421+0.0im
  [2, 3]  =  1.41421+0.0im
  [3, 3]  =  1.5+0.0im
  [4, 4]  =  1.5+0.0im
  [5, 4]  =  1.41421+0.0im
  [4, 5]  =  1.41421+0.0im
  [5, 5]  =  0.5+0.0im
  [6, 6]  =  2.5+0.0im

In [6]:
#  Now I think we are ready to calculate the Zeeman splitting. 
A=1420.405751768                # Hyperfine structure of the ground state of H, A*hbar/(2*pi), in MHz. 
ħ=1.0546*10^(-34)                  # Planck constant in J-s.
c=2.9979*10^(8)                    # Spped of light in m/s.
e=1.6*10^(-19)                  # Electron charge in Coulomb
mₑ=9.11*10^(-31)                 # Electron mass in kg
ϵ₀=8.854*10^(-12)                # Vacuum permitivity
α=(1/(4*pi*ϵ₀))*e^2/(ħ*c)                     # Fine structure constant
ξ₂ₚ=(1/(48*ħ^2))*mₑ*c^2*α^4*ħ/10^6     # radial part of SO interaction for 2p
B₀=range(0,10.0,step=0.01)    # 1.0*10^(-4)     Magnetic field in Tesla
ω₀=-(e/(2*mₑ))*B₀/10^6          # frequency in MHz

0.0:-878.1558726673981:-878155.8726673981

In [7]:
Hlist = [ξ₂ₚ*S1S2 +  (ω/(2*pi)) * (S1z + 2 * S2z ) for ω=ω₀]         # The Hamiltonian is given by eq. (E-8) in Chapter XII.E.1.b. of Cohen-Tannoudji, Quantum Mechanics 
results=[eigenstates(dense(H)) for H=Hlist]; # This is a way of looping over the elements in the array Hlist to get an array called results.


In [8]:
# Here we unpack the Eigenvalues and create 1-dimensional arrays for each Eigenvalue at different magnetic field values.
E1=[results[i][1][1] for i=range(1,length=length(results))]
E2=[results[i][1][2] for i=range(1,length=length(results))]
E3=[results[i][1][3] for i=range(1,length=length(results))]
E4=[results[i][1][4] for i=range(1,length=length(results))]
E5=[results[i][1][5] for i=range(1,length=length(results))]
E6=[results[i][1][6] for i=range(1,length=length(results))];

In [9]:
#  Now we plot each Eigenvalue as a function of magnetic field. 

Plots.plot(B₀,[E1,E2,E3,E4,E5,E6], xaxis="Magnetic Field [T]", yaxis="Energy Shift [MHz]",title="Hydrogen 2p level Zeeman Effect")




<!DOCTYPE html>
 
 
 Plots.jl

In [15]:
# Now let us try to solve the problem again, but in the |Jm> basis. 
# So basically we have the following basis:
# B1=[1,↑]
# B2=[1,↓]
# B3=[0,↑]
# B4=[0,↓]
# B5=[-1,↑]
# B6=[-1,↓]
B1=Ket(m1m2basis,[1;0;0;0;0;0])
B2=Ket(m1m2basis,[0;1;0;0;0;0])
B3=Ket(m1m2basis,[0;0;1;0;0;0])
B4=Ket(m1m2basis,[0;0;0;1;0;0])
B5=Ket(m1m2basis,[0;0;0;0;1;0])
B6=Ket(m1m2basis,[0;0;0;0;1;0])
# We can check using our S1z and S2z operators. 
print("Here are the basis vectors for the |Lz,Sz> basis:\n")
print("B1:   Lz = ",real(dagger(B1)*S1z*B1),"      Sz = ",real(dagger(B1)*S2z*B1), "\n")
print("B2:   Lz = ",real(dagger(B2)*S1z*B2),"      Sz = ",real(dagger(B2)*S2z*B2), "\n")
print("B3:   Lz = ",real(dagger(B3)*S1z*B3),"      Sz = ",real(dagger(B3)*S2z*B3), "\n")
print("B4:   Lz = ",real(dagger(B4)*S1z*B4),"      Sz = ",real(dagger(B4)*S2z*B4), "\n")
print("B5:   Lz = ",real(dagger(B5)*S1z*B5),"      Sz = ",real(dagger(B5)*S2z*B5), "\n")
print("B6:   Lz = ",real(dagger(B6)*S1z*B6),"      Sz = ",real(dagger(B6)*S2z*B6), "\n")

Here are the basis vectors for the |Lz,Sz> basis:
B1:   Lz = 1.0      Sz = 0.5
B2:   Lz = 1.0      Sz = -0.5
B3:   Lz = 0.0      Sz = 0.5
B4:   Lz = 0.0      Sz = -0.5
B5:   Lz = -1.0      Sz = 0.5
B6:   Lz = -1.0      Sz = 0.5


In [16]:
# Now let us try to write the ClebschGordan coefficients to transform from the |Lz,Sz> basis ot the |J,m> basis:
# We will have two possible values of J: 3/2 and 1/2. 
# A good explanation can be  found in the section on Clebsch-Gordan coefficients in
# R. Shankar, Principles of Quantum Mechanics, 2nd edition, New York, Plenum Press, 1994 (ISBN 0-306-44790-8)
# In fact, I found the entire chapter on Addition of Angular momentum (Chapter 15) very helpful.
C=[ clebschgordan(1//1,1//1,1//2,1//2,3//2,3//2)  clebschgordan(1//1,1//1,1//2,-1//2,3//2,3//2)  clebschgordan(1//1,0,1//2,1//2,3//2,3//2)  clebschgordan(1//1,0,1//2,-1//2,3//2,3//2)  clebschgordan(1//1,-1//1,1//2,1//2,3//2,3//2)  clebschgordan(1//1,-1//1,1//2,-1//2,3//2,3//2);
    clebschgordan(1//1,1//1,1//2,1//2,3//2,1//2)  clebschgordan(1//1,1//1,1//2,-1//2,3//2,1//2)  clebschgordan(1//1,0,1//2,1//2,3//2,1//2)  clebschgordan(1//1,0,1//2,-1//2,3//2,1//2)  clebschgordan(1//1,-1//1,1//2,1//2,3//2,1//2)  clebschgordan(1//1,-1//1,1//2,-1//2,3//2,1//2);
    clebschgordan(1//1,1//1,1//2,1//2,3//2,-1//2) clebschgordan(1//1,1//1,1//2,-1//2,3//2,-1//2) clebschgordan(1//1,0,1//2,1//2,3//2,-1//2) clebschgordan(1//1,0,1//2,-1//2,3//2,-1//2) clebschgordan(1//1,-1//1,1//2,1//2,3//2,-1//2) clebschgordan(1//1,-1//1,1//2,-1//2,3//2,-1//2);
    clebschgordan(1//1,1//1,1//2,1//2,3//2,-3//2) clebschgordan(1//1,1//1,1//2,-1//2,3//2,-3//2) clebschgordan(1//1,0,1//2,1//2,3//2,-3//2) clebschgordan(1//1,0,1//2,-1//2,3//2,-3//2) clebschgordan(1//1,-1//1,1//2,1//2,3//2,-3//2) clebschgordan(1//1,-1//1,1//2,-1//2,3//2,-3//2);
    clebschgordan(1//1,1//1,1//2,1//2,1//2,1//2)  clebschgordan(1//1,1//1,1//2,-1//2,1//2,1//2)  clebschgordan(1//1,0,1//2,1//2,1//2,1//2)  clebschgordan(1//1,0,1//2,-1//2,1//2,1//2)  clebschgordan(1//1,-1//1,1//2,1//2,1//2,1//2)  clebschgordan(1//1,-1//1,1//2,-1//2,1//2,1//2);
    clebschgordan(1//1,1//1,1//2,1//2,1//2,-1//2) clebschgordan(1//1,1//1,1//2,-1//2,1//2,-1//2) clebschgordan(1//1,0,1//2,1//2,1//2,-1//2) clebschgordan(1//1,0,1//2,-1//2,1//2,-1//2) clebschgordan(1//1,-1//1,1//2,1//2,1//2,-1//2) clebschgordan(1//1,-1//1,1//2,-1//2,1//2,-1//2)]

6×6 Array{RationalRoots.RationalRoot{BigInt},2}:
 +√(1//1)  +√(0//1)  +√(0//1)  +√(0//1)  +√(0//1)  +√(0//1)
 +√(0//1)  +√(1//3)  +√(2//3)  +√(0//1)  +√(0//1)  +√(0//1)
 +√(0//1)  +√(0//1)  +√(0//1)  +√(2//3)  +√(1//3)  +√(0//1)
 +√(0//1)  +√(0//1)  +√(0//1)  +√(0//1)  +√(0//1)  +√(1//1)
 +√(0//1)  +√(2//3)  -√(1//3)  +√(0//1)  +√(0//1)  +√(0//1)
 +√(0//1)  +√(0//1)  +√(0//1)  +√(1//3)  -√(2//3)  +√(0//1)

In [25]:
# Now let's see what we have. Can we construct the |J,m> basis vectors?
# Let us construct by hand at first. Using the matrix C, we obtain:
Jm1 = B1                                # Jm1 = |3//2,  3//2>
Jm2 = √(1//3) * B2 + √(2//3) * B3       # Jm2 = |3//2,  1//2>
Jm3 = √(2//3) * B4 + √(1//3) * B5       # Jm3 = |3//2, -1//2>
Jm4 = B6                                # Jm4 = |3//2, -3//2>
Jm5 = √(2//3) * B2 - √(1//3) * B3       # Jm5 = |1//2,  1//2>
Jm6 = √(1//3) * B4 - √(2//3) * B5       # Jm6 = |1//2, -1//2>
# We compare our result with Eqns. (36-a) and (36-b) of Cohen-Tannoudji, Aₓ.2.b. (pp. 1033-1034)
# The Clebsch-Gordan coefficients in matrix C seem to be correct, so we define our subspace basis:
Jmbasis=SubspaceBasis([Jm1,Jm2,Jm3,Jm4,Jm5,Jm6])         # Now define the |J,m> basis using these basis vectors

Subspace(superbasis=[Spin(1/2) ⊗ Spin(1)], states:6)

In [26]:
# Next we want to transform the operators for the z-component of the spins, Lz and Sz.

# The elements of C are type RationalRoots.
# We want them to be Float64. 
# We use the vectorized dot-function Float64 
# to convert each element of C to a Float64.
C=Float64.(C)

6×6 Array{Float64,2}:
 1.0  0.0        0.0       0.0        0.0       0.0
 0.0  0.57735    0.816497  0.0        0.0       0.0
 0.0  0.0        0.0       0.816497   0.57735   0.0
 0.0  0.0        0.0       0.0        0.0       1.0
 0.0  0.816497  -0.57735   0.0        0.0       0.0
 0.0  0.0        0.0       0.57735   -0.816497  0.0

In [27]:
# Next we define the transform matrix as an operator. 
# I think the right way to do it, actually, is to define our transform matrix
# as an operator with a left-side basis |F,m>
# and a right-side basis |m₁,m₂>. 
TransformMatrix=Operator(Jmbasis,m1m2basis,C)

Operator(dim=6x6)
  basis left:  Subspace(superbasis=[Spin(1/2) ⊗ Spin(1)], states:6)
  basis right: [Spin(1/2) ⊗ Spin(1)]
 1.0  0.0        0.0       0.0        0.0       0.0
 0.0  0.57735    0.816497  0.0        0.0       0.0
 0.0  0.0        0.0       0.816497   0.57735   0.0
 0.0  0.0        0.0       0.0        0.0       1.0
 0.0  0.816497  -0.57735   0.0        0.0       0.0
 0.0  0.0        0.0       0.57735   -0.816497  0.0

In [29]:
# Now when we take the Hermitian conjugage, the QuantumOptics package
# should handle everything for us. 
TransformMatrixDagger=dagger(TransformMatrix)
# And now we transform our operators:
LzJmbasis=TransformMatrix*(S1z)*TransformMatrixDagger
SzJmbasis=TransformMatrix*(S1z)*TransformMatrixDagger
S1S2Jmbasis=TransformMatrix*(S1S2)*TransformMatrixDagger           
JzJmbasis=LzJmbasis+SzJmbasis            # I think this should be allowed.  

Operator(dim=6x6)
  basis: Subspace(superbasis=[Spin(1/2) ⊗ Spin(1)], states:6)
 2.0+0.0im       0.0+0.0im  …       0.0+0.0im       0.0+0.0im
 0.0+0.0im  0.666667+0.0im     0.942809+0.0im       0.0+0.0im
 0.0+0.0im       0.0+0.0im          0.0+0.0im  0.942809+0.0im
 0.0+0.0im       0.0+0.0im          0.0+0.0im       0.0+0.0im
 0.0+0.0im  0.942809+0.0im      1.33333+0.0im       0.0+0.0im
 0.0+0.0im       0.0+0.0im  …       0.0+0.0im  -1.33333+0.0im

In [33]:
# Now let's see if we can construct the Hamiltonian and obtain the Breit-Rabi diagram using the Jm-basis operators. 
HlistJm = [ξ₂ₚ*S1S2Jmbasis +  (ω/(2*pi)) * (LzJmbasis + 2 * SzJmbasis ) for ω=ω₀]         
results=[eigenstates(dense((HJm+dagger(HJm))/2)) for HJm=HlistJm]; 

In [34]:
# Here we unpack the Eigenvalues and create 1-dimensional arrays for each Eigenvalue at different magnetic field values.
E1Jm=[results[i][1][1] for i=range(1,length=length(results))]
E2Jm=[results[i][1][2] for i=range(1,length=length(results))]
E3Jm=[results[i][1][3] for i=range(1,length=length(results))]
E4Jm=[results[i][1][4] for i=range(1,length=length(results))]
E5Jm=[results[i][1][5] for i=range(1,length=length(results))]
E6Jm=[results[i][1][6] for i=range(1,length=length(results))];
Plots.plot(B₀,[E1Jm,E2Jm,E3Jm,E4Jm,E5Jm,E6Jm],xaxis="Magnetic field B [T]", yaxis="Energy [MHz]", title="Hydrogen 1s level Zeeman Effect")

<!DOCTYPE html>
 
 
 Plots.jl

In [ ]:
# Now we need the J+ and J- operators in the Jm basis. Not quite sure how to do that yet. 
# Then we would just add it to the 